In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as pltpatches
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
import sklearn.metrics as metrics
import kagglehub
import cv2 as cv
import random

#Original dataset
#path = kagglehub.dataset_download("saurabhshahane/mnist-for-object-detection")
#im_path = "/Users/quino070/.cache/kagglehub/datasets/saurabhshahane/mnist-for-object-detection/versions/1/mnist_detection"

In [2]:
im_path = "/Users/quino070/LeafShapeAnalysis/mnist_detection"

In [3]:
#get file names
train_images = sorted(os.listdir(f"{im_path}/train/images"))
test_images = sorted(os.listdir(f"{im_path}/test/images"))

#get background
background1 = cv.imread('/Users/quino070/LeafShapeAnalysis/bg1.png')
background1 = cv.cvtColor(background1, cv.COLOR_BGRA2RGB)
bg1 = background1.copy()
background2 = cv.imread('/Users/quino070/LeafShapeAnalysis/bg2.png')
background2 = cv.cvtColor(background2, cv.COLOR_BGRA2RGB)
bg2 = background2.copy()

In [4]:
def invert_and_bg(dataset, tset, bg1=bg1, bg2=bg2):
    
    for im in dataset:
        #get image and label path
        img = f"{im_path}/{tset}/images/{im}"
        lab_id = im.replace(".png", ".txt")
        label = f"{im_path}/{tset}/labels/{lab_id}"
    
        #invert image and replace original
        overlay = cv.imread(img)
        inv = cv.bitwise_not(overlay) 

        #Add background 1
        _, mask = cv.threshold(cv.cvtColor(overlay, cv.COLOR_BGR2GRAY), 
                               0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

        bg1 = cv.resize(bg1, (inv.shape[1], inv.shape[0]))
        img_with_bg1 = bg1.copy()
        img_with_bg1[mask == 255] = inv[mask == 255]
        img_with_bg1 = cv.cvtColor(img_with_bg1, cv.COLOR_BGRA2RGB)
        
        #Add background 2
        bg2 = cv.resize(bg2, (inv.shape[1], inv.shape[0]))
        img_with_bg2 = bg2.copy()
        img_with_bg2[mask == 255] = inv[mask == 255]
        img_with_bg2 = cv.cvtColor(img_with_bg2, cv.COLOR_BGRA2RGB)

        #Save
        #1. Inverted image
        cv.imwrite(img, inv)
        #2.a Image with background 1
        cv.imwrite(img.replace(".png", "_1.png"), img_with_bg1)
        #2.b Copy label file
        df = pd.read_csv(label)
        df.to_csv(label.replace(".txt", "_1.txt"), index=False)
        #3.a Image with background 2
        cv.imwrite(img.replace(".png", "_2.png"), img_with_bg2)
        #3.b Copy label file
        df.to_csv(label.replace(".txt", "_2.txt"), index=False)

In [5]:
invert_and_bg(train_images, "train")
invert_and_bg(test_images, "test")